In [14]:
# 필요한 module import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# Random Seed를 위해서 포함
import os
import random

# sklearn
from sklearn import linear_model  # LinearRegression 모델이 이 안에 들어 있어요!
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TensorFlow
import tensorflow as tf   # random seed 때문에 필요!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# 시드 설정하는 함수
def seed_everything(seed=42):

    # 1. 환경변수
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 2. Python, Numpy
    random.seed(seed)
    np.random.seed(seed)

    # 3. TensorFlow
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    tf.random.set_seed(seed)

    # 4. PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)  # PyTorch 1.8+ 권장

    print(f"Seeds set to {seed}")

seed_everything(42)

Seeds set to 42


In [15]:
df = pd.read_csv('./data/train.csv')

print(f"행 : {df.shape[0]}, 열 : {df.shape[1]}")

print(df.isnull().sum())

행 : 42000, 열 : 785
label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64


In [16]:
# 학습데이터 생성
x_data = df.drop('label', axis=1).values
y_data = df['label'].values.reshape(-1,1)

In [17]:
# 데이터 분할
x_data_train, x_data_test, y_data_train, y_data_test = \
train_test_split(x_data,
                 y_data,
                 test_size=0.3,
                 stratify=y_data,
                 shuffle=True,
                 random_state=42)

In [18]:
# 7. 정규화
scaler_x = StandardScaler()
scaler_x.fit(x_data_train)

x_data_train_norm = scaler_x.transform(x_data_train)
x_data_test_norm = scaler_x.transform(x_data_test)

In [19]:
# sklearn 구현(데이터 누수)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Model 생성
knn = KNeighborsClassifier(
    n_neighbors=3,      # 참고할 이웃 수
    weights='distance', # 거리 기반 가중치
    n_jobs=-1           # 병렬처리 (CPU 전체 사용)
)

cv = StratifiedKFold(n_splits=5,
                     shuffle=True,
                     random_state=42)

sklearn_score = cross_val_score(estimator=knn,
                                X=x_data_train_norm,
                                y=y_data_train,
                                scoring='accuracy',
                                cv=cv)

/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A col

In [29]:
print(sklearn_score)
print(sklearn_score.mean())

[0.93163265 0.93707483 0.93452381 0.93707483 0.93214286]
0.9344897959183672


In [30]:
# 모델 학습
knn.fit(x_data_train_norm,
                  y_data_train)

/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_jobs=-1, n_neighbors=3, weights='distance')

In [31]:
# Evaluation
sklearn_y_pred = knn.predict(x_data_test_norm)
sklearn_y_pred


print(classification_report(y_data_test.ravel(),
                            sklearn_y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1240
           1       0.95      0.99      0.97      1405
           2       0.95      0.94      0.94      1253
           3       0.92      0.93      0.93      1305
           4       0.94      0.92      0.93      1222
           5       0.92      0.91      0.92      1139
           6       0.95      0.98      0.96      1241
           7       0.93      0.93      0.93      1320
           8       0.96      0.87      0.91      1219
           9       0.90      0.92      0.91      1256

    accuracy                           0.94     12600
   macro avg       0.94      0.94      0.94     12600
weighted avg       0.94      0.94      0.94     12600



In [32]:
# sklearn 구현 (데이터 누수 제거)
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

pipe = make_pipeline(
    StandardScaler(),
    linear_model.LogisticRegression(max_iter=1000)
)

cv = StratifiedKFold(n_splits=5,
                     shuffle=True,
                     random_state=42)


# StandardScaler가 각 폴드의 훈련 데이터에 대해서만 fit.
sklearn_score = cross_val_score(estimator=pipe,
                                X=x_data_train,
                                y=y_data_train,
                                scoring='accuracy',
                                cv=cv)
print(sklearn_score)
print(sklearn_score.mean())

/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-v

[0.89132653 0.90085034 0.89642857 0.89081633 0.89914966]
0.8957142857142857


In [34]:
# 모델 학습
pipe.fit(x_data_train,
         y_data_train)

# Evaluation
sklearn_y_pred = pipe.predict(x_data_test)
print(sklearn_y_pred)

print(classification_report(y_data_test.ravel(),
                            sklearn_y_pred))

/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[3 0 8 ... 8 0 3]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1240
           1       0.94      0.97      0.95      1405
           2       0.88      0.88      0.88      1253
           3       0.89      0.87      0.88      1305
           4       0.91      0.91      0.91      1222
           5       0.86      0.84      0.85      1139
           6       0.93      0.95      0.94      1241
           7       0.92      0.93      0.92      1320
           8       0.87      0.84      0.85      1219
           9       0.88      0.89      0.88      1256

    accuracy                           0.90     12600
   macro avg       0.90      0.90      0.90     12600
weighted avg       0.90      0.90      0.90     12600



In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import datetime

# 모델 생성
keras_model = Sequential()

# 입력층: (784,) — 28x28 이미지를 1차원으로 Flatten한 형태
keras_model.add(Input(shape=(784,)))

# 은닉층 0: 256 뉴런, ReLU 활성화
keras_model.add(Dense(units=256, activation='relu'))

# 은닉층 1: 128 뉴런, ReLU 활성화
keras_model.add(Dense(units=128, activation='relu'))

# 은닉층 2: 64 뉴런, ReLU 활성화
keras_model.add(Dense(units=64, activation='relu'))

# 출력층: 10개 클래스 (숫자 0~9), Softmax 활성화
keras_model.add(Dense(units=10, activation='softmax'))


# 컴파일
keras_model.compile(
    optimizer=Adam(learning_rate=1e-3),                   # Adam optimizer (lr=0.001 권장)
    loss='sparse_categorical_crossentropy',               # 정수형 라벨용 다중분류 손실함수
    metrics=['accuracy']                                  # 정확도 계산
)

log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tb_cb = TensorBoard(log_dir=log_dir, histogram_freq=1)


# 모델 학습
keras_model.fit(
    x_data_train_norm,     # 정규화된 학습 데이터
    y_data_train,          # 학습 라벨
    epochs=100,             # 학습 반복
    batch_size=128,        # 배치 크기
    validation_split=0.2,  # 학습 데이터 중 20%를 검증용으로 자동 분리
    callbacks=[tb_cb],     # TensorBoard 로그 기록
    verbose=0              # 학습 과정 출력
)


# 평가

test_loss, test_acc = keras_model.evaluate(x_data_test_norm, y_data_test)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")


394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9699 - loss: 0.3421

✅ Test Accuracy: 0.9699


In [40]:
# Evaluation

keras_result = keras_model.evaluate(x_data_test_norm,
                                    y_data_test)
print(keras_result)

keras_y_pred = keras_model.predict(x_data_test_norm)
keras_y_class = keras_y_pred.argmax(axis=1)

print(classification_report(y_data_test.ravel(),
                            keras_y_class))

394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9699 - loss: 0.3421
[0.3420718312263489, 0.9699206352233887]
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1240
           1       0.98      0.99      0.99      1405
           2       0.97      0.97      0.97      1253
           3       0.96      0.95      0.96      1305
           4       0.97      0.97      0.97      1222
           5       0.95      0.96      0.95      1139
           6       0.98      0.99      0.98      1241
           7       0.97      0.98      0.97      1320
           8       0.97      0.95      0.96      1219
           9       0.96      0.96      0.96      1256

    accuracy                           0.97     12600
   macro avg       0.97      0.97      0.97     12600
weighted avg       0.97      0.97      0.97     12600



In [42]:
# PyTorch 구현

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x_tensor_train = torch.FloatTensor(x_data_train_norm).to(device)
x_tensor_test = torch.FloatTensor(x_data_test_norm).to(device)

y_tensor_train = torch.LongTensor(y_data_train.ravel()).to(device)
y_tensor_test = torch.LongTensor(y_data_test.ravel()).to(device)

class MulticlassModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 256),     # 입력 784차원 (28x28)
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)       # 출력 10개 (0~9)
        )

    def forward(self, x):
        return self.net(x)

torch_model = MulticlassModel().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(torch_model.parameters(),
                       lr=5e-4)

epochs = 100

for epoch in range(epochs):

    y_pred = torch_model(x_tensor_train)

    loss = criterion(y_pred, y_tensor_train)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if epoch % 10 == 0:
        print(f'epochs({epoch}/{epochs}), Loss : {loss.item()}')

with torch.no_grad():
    torch_y_pred = torch_model(x_tensor_test)
    torch_y_pred_class = torch.argmax(torch_y_pred,
                                      dim=1).cpu().numpy()

print(classification_report(y_data_test.ravel(),
                            torch_y_pred_class))



epochs(0/100), Loss : 2.304212808609009
epochs(10/100), Loss : 1.714231252670288
epochs(20/100), Loss : 0.9769160151481628
epochs(30/100), Loss : 0.5443826913833618
epochs(40/100), Loss : 0.3734297752380371
epochs(50/100), Loss : 0.29405125975608826
epochs(60/100), Loss : 0.2446979284286499
epochs(70/100), Loss : 0.2106665074825287
epochs(80/100), Loss : 0.18545615673065186
epochs(90/100), Loss : 0.16525845229625702
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1240
           1       0.96      0.98      0.97      1405
           2       0.95      0.94      0.95      1253
           3       0.94      0.91      0.92      1305
           4       0.94      0.94      0.94      1222
           5       0.93      0.92      0.92      1139
           6       0.95      0.98      0.97      1241
           7       0.94      0.95      0.95      1320
           8       0.92      0.90      0.91      1219
           9       0.92      0.91      

# 테스트

# skitlearn

In [43]:
test_df = pd.read_csv("./data/test.csv")

x_data_test_final = test_df.values / 255.0
sklearn_y_pred_final = knn.predict(x_data_test_final)

print("예측 결과 샘플:", sklearn_y_pred_final[:10])

submission = pd.DataFrame({
    "ImageId": np.arange(1, len(sklearn_y_pred_final) + 1),
    "Label": sklearn_y_pred_final
})

# CSV 저장
submission.to_csv("submission_sklearn.csv", index=False)
print("submission_sklearn.csv 파일 생성 완료!")

예측 결과 샘플: [2 3 9 9 3 9 0 3 5 3]
submission_sklearn.csv 파일 생성 완료!


## Tensorflow

In [44]:
x_data_test_final = test_df.values / 255.0
keras_y_pred = keras_model.predict(x_data_test_final)

keras_y_class = keras_y_pred.argmax(axis=1)
print("예측 결과 샘플:", keras_y_class[:10])

submission = pd.DataFrame({
    "ImageId": np.arange(1, len(keras_y_class) + 1),
    "Label": keras_y_class
})

# CSV로 저장
submission.to_csv("submission_keras.csv", index=False)
print("submission_keras.csv 파일 생성 완료!")

875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
예측 결과 샘플: [2 0 9 9 3 9 0 3 0 3]
submission_keras.csv 파일 생성 완료!


## PyTorch

In [46]:
x_data_test_final = test_df.values / 255.0
x_tensor_test_final = torch.FloatTensor(x_data_test_final).to(device)

torch_model.eval()  # 평가 모드
with torch.no_grad():
    y_pred_logits = torch_model(x_tensor_test_final)
    y_pred_class = torch.argmax(y_pred_logits, dim=1).cpu().numpy()

print("예측 결과 샘플:", y_pred_class[:10])

submission = pd.DataFrame({
    "ImageId": np.arange(1, len(y_pred_class) + 1),
    "Label": y_pred_class
})
submission.to_csv("submission_pytorch.csv", index=False)

예측 결과 샘플: [2 0 8 9 3 7 0 3 0 3]
